In [7]:
pip install dotenv

  Using cached dotenv-0.9.9-py2.py3-none-any.whl.metadata (279 bytes)
  Using cached python_dotenv-1.1.0-py3-none-any.whl.metadata (24 kB)
Using cached dotenv-0.9.9-py2.py3-none-any.whl (1.9 kB)
Using cached python_dotenv-1.1.0-py3-none-any.whl (20 kB)
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2/2 [dotenv]
Note: you may need to restart the kernel to use updated packages.


In [8]:
from dotenv import load_dotenv

load_dotenv("GLM-Z1-9B-0414.env")  # 指定要加载的 .env 文件

True

In [4]:
import os
print("当前工作目录：", os.getcwd())

当前工作目录： /Users/yukun/Desktop/mda/主观分析预处理/主观分析预处理代码


In [9]:
print("API_KEY:", os.getenv("API_KEY"))
print("MODEL_NAME:", os.getenv("MODEL_NAME"))

API_KEY: sk-vtpefpeaoggdppvaxlhmwmidtsevuryhxfubpsztrnhgvmmn
MODEL_NAME: THUDM/GLM-Z1-9B-0414


In [6]:
import os
import requests
import pandas as pd
import time
import json
from concurrent.futures import ThreadPoolExecutor, as_completed
from dotenv import load_dotenv  # ✅ 加载 .env 所需

# ✅ 加载环境变量（确保你本地有 model.env 文件）
load_dotenv("GLM-Z1-9B-0414.env")  # 指定文件名

# ✅ 从环境变量中获取 API_KEY 和模型名
api_key = os.getenv("API_KEY")
model_name = os.getenv("MODEL_NAME")

if not api_key or not model_name:
    raise ValueError("❌ 缺少 API_KEY 或 MODEL_NAME，请检查 model.env 文件是否设置正确")

url = "https://api.siliconflow.cn/v1/chat/completions"
headers = {
    "Content-Type": "application/json",
    "Authorization": f"Bearer {api_key}"
}
# ✅ 输入输出路径
input_folder = "/Users/yukun/Desktop/mda/MDA训练集"
output_csv_path = "519GLM-Z1-9B-0414.env.csv"

# ✅ 提取字段
output_fields = [
    "文件名",
    "董事会成员变动",
    "董秘变更",
    "高级管理层变更",
    "重大资产重组",
    "控股股东/实际控制人变更",
    "子公司或重要分支机构重大变化",
    "对财务状况的分析",
    "营业收入增长情况",
    "净利润变动分析",
    "毛利率变动分析",
    "销售费用率变化",
    "管理费用控制情况",
    "经营性现金流变化趋势",
    "应收账款变化与风险",
    "资产负债结构调整",
    "利润质量分析（是否依赖非经常性损益）",
    "核心竞争力分析（品牌、成本、效率等）",
    "市场拓展与业务布局进展",
    "区域/产品结构优化情况",
    "供应链/渠道建设进展",
    "战略合作与外部联盟情况",
    "投资项目执行及回报分析",
    "企业文化建设与团队稳定性",
    "对标管理与精细化运营成果",
    "技术创新与研发投入",
    "新产品/新技术推出情况",
    "研发费用占比变化",
    "专利数量与知识产权布局",
    "智能制造/数字化转型进展",
    "行业竞争格局分析",
    "行业发展趋势与机会识别",
    "国家政策影响与公司应对",
    "未来经营计划与增长目标",
    "订单/客户/签约情况展望",
    "海外市场/出口业务计划",
    "对经济周期或宏观环境的判断",
    "面临的主要风险因素及变化",
    "政策/监管风险与公司响应",
    "环保合规/碳排放压力",
    "土地/原材料/人工成本波动风险",
    "资金链紧张预警",
    "核心客户集中度及流失风险",
    "可持续发展战略与目标",
    "社会责任履行与公益活动",
    "节能降耗/绿色生产措施"
]

# ✅ Prompt模板
def build_prompt(text):
    return f"""
以下是某公司年报中的“管理层讨论与分析”部分内容，请你遵循以下要求处理该文本：

1. 删除冗余性强的空话、套话。
2. 仅保留具有研究价值的具体内容，尤其是与公司治理和业绩相关的信息。
3. 请严格按照以下结构模板输出内容，每个条目都必须出现，即使原文没有相关内容，也请写明“无相关信息”或“未提及”。

【固定输出结构如下】：
{chr(10).join([f"- 【{field}】：" for field in output_fields[1:]])}

原文如下：
{text}
"""

# ✅ 每个文件的处理函数
def process_file(file_path):
    filename = os.path.basename(file_path)
    try:
        with open(file_path, "r", encoding="utf-8") as f:
            text = f.read()

        prompt = build_prompt(text)
        data = {
            "model": model_name,
            "messages": [{"role": "user", "content": prompt}],
            "temperature": 0.2,
            "max_tokens": 2048
        }

        response = requests.post(url, headers=headers, json=data)
        response.raise_for_status()
        response_data = response.json()

        if "choices" not in response_data:
            raise ValueError("接口返回中未包含choices字段，可能是模型名称错误或token用尽")

        content = response_data["choices"][0]["message"]["content"]
        row = [filename]

        for field in output_fields[1:]:
            key = f"- 【{field}】："
            start = content.find(key)
            if start != -1:
                end = content.find("- 【", start + 1)
                value = content[start + len(key):end].strip() if end != -1 else content[start + len(key):].strip()
            else:
                value = "未提及"
            row.append(value)

        print(f"✔ 完成：{filename}")
        return row

    except Exception as e:
        print(f"✘ 出错：{filename}，错误：{e}")
        return [filename] + ["ERROR"] * (len(output_fields) - 1)

# ✅ 并发运行
def run_parallel():
    file_paths = [
        os.path.join(input_folder, f)
        for f in os.listdir(input_folder)
        if f.endswith(".txt")
    ]

    results = []

    with ThreadPoolExecutor(max_workers=5) as executor:  # 控制并发线程数
        future_to_file = {executor.submit(process_file, file): file for file in file_paths}

        for future in as_completed(future_to_file):
            result = future.result()
            results.append(result)

    df = pd.DataFrame(results, columns=output_fields)
    df.to_csv(output_csv_path, index=False, encoding="utf-8-sig")
    print(f"\n✅ 全部处理完成，结果保存在：{output_csv_path}")

if __name__ == "__main__":
    run_parallel()

✔ 完成：2429-兆驰股份-2021.txt
✔ 完成：895-双汇发展-2021.txt
✔ 完成：2475-立讯精密-2021.txt
✔ 完成：600566-济川药业-2021.txt
✔ 完成：963-华东医药-2021.txt
✔ 完成：2552-宝鼎科技-2021.txt
✔ 完成：300339-润和软件-2021.txt
✔ 完成：600031-三一重工-2021.txt
✔ 完成：2415-海康威视-2021.txt
✔ 完成：2241-歌尔股份-2021.txt
✔ 完成：600276-恒瑞医药-2021.txt
✔ 完成：425-徐工机械-2021.txt
✔ 完成：2507-涪陵榨菜-2021.txt
✔ 完成：600519-贵州茅台-2021.txt
✘ 出错：2230-科大讯飞-2021.txt，错误：HTTPSConnectionPool(host='api.siliconflow.cn', port=443): Max retries exceeded with url: /v1/chat/completions (Caused by ProxyError('Unable to connect to proxy', RemoteDisconnected('Remote end closed connection without response')))

✅ 全部处理完成，结果保存在：519GLM-Z1-9B-0414.env.csv


In [ ]:
# 导入标准库和第三方库
import os
import requests  # 用于发送 HTTP 请求
import pandas as pd  # 用于数据处理与保存为 CSV
import time
import json
from concurrent.futures import ThreadPoolExecutor, as_completed  # 并发处理
from dotenv import load_dotenv  # ✅ 用于加载 .env 文件中的环境变量（如 API_KEY）

# ✅ 加载环境变量文件，确保其中包含 API_KEY 和 MODEL_NAME
load_dotenv("GLM-Z1-9B-0414.env")  # 指定 .env 文件路径

# ✅ 从环境变量中读取模型相关配置
api_key = os.getenv("API_KEY")
model_name = os.getenv("MODEL_NAME")

# ✅ 如果未能正确读取关键变量，则报错
if not api_key or not model_name:
    raise ValueError("❌ 缺少 API_KEY 或 MODEL_NAME，请检查 model.env 文件是否设置正确")

# ✅ 模型 API 地址与请求头
url = "https://api.siliconflow.cn/v1/chat/completions"
headers = {
    "Content-Type": "application/json",
    "Authorization": f"Bearer {api_key}"
}

# ✅ 输入输出路径配置
input_folder = "/Users/yukun/Desktop/mda/MDA训练集"  # 年报txt文件所在路径
output_csv_path = "519GLM-Z1-9B-0414.env.csv"  # 输出的CSV文件路径

# ✅ 所需提取的字段名，首列为“文件名”，其余为分析内容标签
output_fields = [
    "文件名",
    "董事会成员变动",
    "董秘变更",
    "高级管理层变更",
    "重大资产重组",
    "控股股东/实际控制人变更",
    "子公司或重要分支机构重大变化",
    "对财务状况的分析",
    "营业收入增长情况",
    "净利润变动分析",
    "毛利率变动分析",
    "销售费用率变化",
    "管理费用控制情况",
    "经营性现金流变化趋势",
    "应收账款变化与风险",
    "资产负债结构调整",
    "利润质量分析（是否依赖非经常性损益）",
    "核心竞争力分析（品牌、成本、效率等）",
    "市场拓展与业务布局进展",
    "区域/产品结构优化情况",
    "供应链/渠道建设进展",
    "战略合作与外部联盟情况",
    "投资项目执行及回报分析",
    "企业文化建设与团队稳定性",
    "对标管理与精细化运营成果",
    "技术创新与研发投入",
    "新产品/新技术推出情况",
    "研发费用占比变化",
    "专利数量与知识产权布局",
    "智能制造/数字化转型进展",
    "行业竞争格局分析",
    "行业发展趋势与机会识别",
    "国家政策影响与公司应对",
    "未来经营计划与增长目标",
    "订单/客户/签约情况展望",
    "海外市场/出口业务计划",
    "对经济周期或宏观环境的判断",
    "面临的主要风险因素及变化",
    "政策/监管风险与公司响应",
    "环保合规/碳排放压力",
    "土地/原材料/人工成本波动风险",
    "资金链紧张预警",
    "核心客户集中度及流失风险",
    "可持续发展战略与目标",
    "社会责任履行与公益活动",
    "节能降耗/绿色生产措施"
]

# ✅ 构建提问模板（Prompt），用于清洗并结构化模型输出
def build_prompt(text):
    return f"""
以下是某公司年报中的“管理层讨论与分析”部分内容，请你遵循以下要求处理该文本：

1. 删除冗余性强的空话、套话。
2. 仅保留具有研究价值的具体内容，尤其是与公司治理和业绩相关的信息。
3. 请严格按照以下结构模板输出内容，每个条目都必须出现，即使原文没有相关内容，也请写明“无相关信息”或“未提及”。

【固定输出结构如下】：
{chr(10).join([f"- 【{field}】：" for field in output_fields[1:]])}

原文如下：
{text}
"""

# ✅ 定义处理每个单独文件的函数
def process_file(file_path):
    filename = os.path.basename(file_path)  # 获取文件名
    try:
        # 读取txt文件内容
        with open(file_path, "r", encoding="utf-8") as f:
            text = f.read()

        # 构建prompt
        prompt = build_prompt(text)

        # 构建API请求参数
        data = {
            "model": model_name,
            "messages": [{"role": "user", "content": prompt}],
            "temperature": 0.2,  # 控制生成的稳定性
            "max_tokens": 2048   # 控制最大返回字数
        }

        # 发送POST请求
        response = requests.post(url, headers=headers, json=data)
        response.raise_for_status()  # 请求失败会抛出异常
        response_data = response.json()

        # 验证返回结构
        if "choices" not in response_data:
            raise ValueError("接口返回中未包含choices字段，可能是模型名称错误或token用尽")

        # 获取返回内容
        content = response_data["choices"][0]["message"]["content"]
        row = [filename]  # 初始化数据行

        # 逐字段提取信息（基于结构化标签）
        for field in output_fields[1:]:
            key = f"- 【{field}】："
            start = content.find(key)
            if start != -1:
                end = content.find("- 【", start + 1)
                value = content[start + len(key):end].strip() if end != -1 else content[start + len(key):].strip()
            else:
                value = "未提及"
            row.append(value)

        print(f"✔ 完成：{filename}")
        return row

    except Exception as e:
        # 异常处理：记录错误并返回 ERROR
        print(f"✘ 出错：{filename}，错误：{e}")
        return [filename] + ["ERROR"] * (len(output_fields) - 1)

# ✅ 并发运行函数，使用多线程同时处理多个文件
def run_parallel():
    # 收集所有 txt 文件路径
    file_paths = [
        os.path.join(input_folder, f)
        for f in os.listdir(input_folder)
        if f.endswith(".txt")
    ]

    results = []  # 所有处理结果列表

    # 使用 ThreadPoolExecutor 并发处理（最多5个线程）
    with ThreadPoolExecutor(max_workers=5) as executor:
        # 提交任务
        future_to_file = {executor.submit(process_file, file): file for file in file_paths}

        # 等待任务完成并收集结果
        for future in as_completed(future_to_file):
            result = future.result()
            results.append(result)

    # 将结果写入 DataFrame，再导出为 CSV
    df = pd.DataFrame(results, columns=output_fields)
    df.to_csv(output_csv_path, index=False, encoding="utf-8-sig")
    print(f"\n✅ 全部处理完成，结果保存在：{output_csv_path}")

# ✅ 主函数入口
if __name__ == "__main__":
    run_parallel()